# Abstract

# Environment

In [38]:
import json
import jsonschema
import os.path as path
import re

In [40]:
import jwst.associations as associations
from jwst.associations import load_asn
from jwst.associations.lib.rules_level2_base import _REGEX_LEVEL2A

# Library

In [46]:
def lv2_v1_to_v2(asn):
    new_asn = dict(asn)
    products = []
    members_to_add = []
    for member in asn['members']:
        product = {
            'name': product_name(member['expname'])
        }
        new_members = []
        new_members.append({
            'expname': member['expname'],
            'exptype': member['exptype']
        })
        for bkg in member.get('bkgexps', []):
            new_members.append({
                'expname': bkg['expname'],
                'exptype': 'BACKGROUND'
            })
            members_to_add.append({
                'expname': bkg['expname'],
                'exptype': 'SCIENCE'
            })
        product['members'] = new_members
        products.append(product)
    
    for new_member in members_to_add:
        product = {
            'name': product_name(new_member['expname'])
        }
        product['members'] = [{
            'expname': new_member['expname'],
            'exptype': 'SCIENCE'
        }]
        products.append(product)
        
    new_asn['products'] = products
    del new_asn['members']
    return new_asn
    

In [25]:
def product_name(expname):
    name = path.basename(expname.lower())
    match = re.match(_REGEX_LEVEL2A, name)
    if match:
        return match.groupdict()['path']
    else:
        return name

# Main

In [17]:
with open('data/asn_level2_v1.json') as fp:
    lv2_v1 = json.load(fp)

In [26]:
lv2_v1

{'asn_id': 'a3002',
 'asn_pool': 'jw82600_001_20160304T145416_pool',
 'asn_rule': 'Asn_MIRI_BkgChop',
 'asn_type': 'background',
 'members': [{'bkgexps': [{'expname': 'test_lrsbkg_rate.fits'}],
   'expname': 'test_lrs1_rate.fits',
   'exptype': 'SCIENCE'},
  {'bkgexps': [{'expname': 'test_lrsbkg_rate.fits'}],
   'expname': 'test_lrs2_rate.fits',
   'exptype': 'SCIENCE'},
  {'bkgexps': [{'expname': 'test_lrsbkg_rate.fits'}],
   'expname': 'test_lrs3_rate.fits',
   'exptype': 'SCIENCE'},
  {'bkgexps': [{'expname': 'test_lrsbkg_rate.fits'}],
   'expname': 'test_lrs4_rate.fits',
   'exptype': 'SCIENCE'}],
 'program': '82600',
 'targname': '2MASS-J00243075-6635527'}

In [47]:
lv2_v2 = lv2_v1_to_v2(lv2_v1)

In [48]:
lv2_v2

{'asn_id': 'a3002',
 'asn_pool': 'jw82600_001_20160304T145416_pool',
 'asn_rule': 'Asn_MIRI_BkgChop',
 'asn_type': 'background',
 'products': [{'members': [{'expname': 'test_lrs1_rate.fits',
     'exptype': 'SCIENCE'},
    {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
   'name': 'test_lrs1'},
  {'members': [{'expname': 'test_lrs2_rate.fits', 'exptype': 'SCIENCE'},
    {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
   'name': 'test_lrs2'},
  {'members': [{'expname': 'test_lrs3_rate.fits', 'exptype': 'SCIENCE'},
    {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
   'name': 'test_lrs3'},
  {'members': [{'expname': 'test_lrs4_rate.fits', 'exptype': 'SCIENCE'},
    {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
   'name': 'test_lrs4'},
  {'members': [{'expname': 'test_lrsbkg_rate.fits', 'exptype': 'SCIENCE'}],
   'name': 'test_lrsbkg'},
  {'members': [{'expname': 'test_lrsbkg_rate.fits', 'exptype': 'SCIENCE'}],
   'name': 

In [49]:
with open('/tmp/level2b_asn.json', 'w') as fp:
    json.dump(lv2_v2, fp)

In [50]:
with open('/tmp/level2b_asn.json') as fp:
    asn = load_asn(fp)

In [51]:
asn

{'asn_id': 'a3002',
 'asn_pool': 'jw82600_001_20160304T145416_pool',
 'asn_rule': 'Asn_MIRI_BkgChop',
 'asn_type': 'background',
 'products': [{'members': [{'expname': 'test_lrs1_rate.fits',
     'exptype': 'SCIENCE'},
    {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
   'name': 'test_lrs1'},
  {'members': [{'expname': 'test_lrs2_rate.fits', 'exptype': 'SCIENCE'},
    {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
   'name': 'test_lrs2'},
  {'members': [{'expname': 'test_lrs3_rate.fits', 'exptype': 'SCIENCE'},
    {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
   'name': 'test_lrs3'},
  {'members': [{'expname': 'test_lrs4_rate.fits', 'exptype': 'SCIENCE'},
    {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
   'name': 'test_lrs4'},
  {'members': [{'expname': 'test_lrsbkg_rate.fits', 'exptype': 'SCIENCE'}],
   'name': 'test_lrsbkg'},
  {'members': [{'expname': 'test_lrsbkg_rate.fits', 'exptype': 'SCIENCE'}],
   'name': 

In [52]:
asn['products']

[{'members': [{'expname': 'test_lrs1_rate.fits', 'exptype': 'SCIENCE'},
   {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
  'name': 'test_lrs1'},
 {'members': [{'expname': 'test_lrs2_rate.fits', 'exptype': 'SCIENCE'},
   {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
  'name': 'test_lrs2'},
 {'members': [{'expname': 'test_lrs3_rate.fits', 'exptype': 'SCIENCE'},
   {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
  'name': 'test_lrs3'},
 {'members': [{'expname': 'test_lrs4_rate.fits', 'exptype': 'SCIENCE'},
   {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}],
  'name': 'test_lrs4'},
 {'members': [{'expname': 'test_lrsbkg_rate.fits', 'exptype': 'SCIENCE'}],
  'name': 'test_lrsbkg'},
 {'members': [{'expname': 'test_lrsbkg_rate.fits', 'exptype': 'SCIENCE'}],
  'name': 'test_lrsbkg'},
 {'members': [{'expname': 'test_lrsbkg_rate.fits', 'exptype': 'SCIENCE'}],
  'name': 'test_lrsbkg'},
 {'members': [{'expname': 'test_lrsbkg_rate.f

In [53]:
asn['products'][0]['members']

[{'expname': 'test_lrs1_rate.fits', 'exptype': 'SCIENCE'},
 {'expname': 'test_lrsbkg_rate.fits', 'exptype': 'BACKGROUND'}]